<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#새로-띄워진-창에서-로그인을-한다" data-toc-modified-id="새로-띄워진-창에서-로그인을-한다-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>새로 띄워진 창에서 로그인을 한다</a></span><ul class="toc-item"><li><span><a href="#댓글-긁어오기-함수" data-toc-modified-id="댓글-긁어오기-함수-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>댓글 긁어오기 함수</a></span></li><li><span><a href="#복습테스트-함수" data-toc-modified-id="복습테스트-함수-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>복습테스트 함수</a></span></li><li><span><a href="#Free-Rider-함수" data-toc-modified-id="Free-Rider-함수-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Free Rider 함수</a></span></li></ul></li><li><span><a href="#excel--불러와서-한-시트에-합치기" data-toc-modified-id="excel--불러와서-한-시트에-합치기-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>excel  불러와서 한 시트에 합치기</a></span><ul class="toc-item"><li><span><a href="#pds-함수-(옛날거)" data-toc-modified-id="pds-함수-(옛날거)-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>pds 함수 (옛날거)</a></span></li></ul></li></ul></div>

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display
import time
import requests
import re
import glob
import os



driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(10)
driver.get('https://nid.naver.com/oauth2.0/authorize?svctype=0&response_type=code&client_id=C9hwybENgOtF&state=IWK3A3NUJQVTFVU2KJPK2DRAZ6L7OXGBFYZHHTH3GMTTXKI6A6ALVCF6SFNE3OLHQ4JVWLE6HFC7Q===&redirect_url=https%3A%2F%2Fauth.band.us%2Fexternal_account_login%3Ftype%3Dnaver')


# 새로 띄워진 창에서 로그인을 한다

## 댓글 긁어오기 함수

In [79]:
def comment(p, v):
    
    
    pdss = [f'pds{i}회차' for i in range(1, p+1)]
    vocas = [f'단어시험0{i}회' for i in range(1, v+1)]
    
    for class_id in list(class_dic.keys()):
        
        df = {}
        
        
        for pds in pdss:
            
            url = class_dic[class_id] + f'/hashtag/{pds}'
            driver.get(url)
            time.sleep(1)

            try: 
                driver.find_element_by_class_name('commentToggle._commentToggleBtn').click()
                time.sleep(0.5)
        

                for _ in range(3):
                    try:
                        driver.find_element_by_class_name('prevComment').click()
                        time.sleep(0.2)
                    except :
                        pass

                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                students = soup.select('.itemWrap')

                student_data = {}

                for student in students :

                    if class_id == 'PURPLE':    # 퍼플은 반명을 한글로 적는 바람에 따로 해줘야 함 
                        name = student.select('button > strong')[0].text
                        name = name[5:].strip() # 전부 다 이거로 하게 되면 띄어쓰기 없는 애들은 솔팅 안됨 
                    else :
                        name = student.select('button > strong')[0].text

                    score = "O"

                    student_data[f'{name}'] = score 

                df[f'{pds}'] = pd.Series(student_data)

    #             print(len(student_data))
            
            except:
                pass
        
        
        for voca in vocas:
            
            url = class_dic[class_id] + f'/hashtag/{voca}'
            driver.get(url)
            time.sleep(1)

            try: 
                driver.find_element_by_class_name('commentToggle._commentToggleBtn').click()
                time.sleep(0.5)
        

                for _ in range(3):
                    try:
                        driver.find_element_by_class_name('prevComment').click()
                        time.sleep(0.2)
                    except :
                        pass

                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                students = soup.select('.itemWrap')

                student_data = {}

                for student in students :

                    if class_id == 'PURPLE':    # 퍼플은 반명을 한글로 적는 바람에 따로 해줘야 함 
                        name = student.select('button > strong')[0].text
                        name = name[5:].strip() # 전부 다 이거로 하게 되면 띄어쓰기 없는 애들은 솔팅 안됨 
                    else :
                        name = student.select('button > strong')[0].text
                    try: 
                        score = int(student.select('p')[0].text)
                    except :
                        score = ""
                        
                    student_data[f'{name}'] = score 

                df[f'{voca}'] = pd.Series(student_data)
            
            except:
                pass


        total = pd.DataFrame(df).reset_index()
        total['index'] = total['index'].str.extract(r'([가-힣].*)') # 한글로 시작하는 시점부터 뒤까지 전부 불러와라 
        total = total.rename(columns={'index':"name"})
        total = total.fillna("X")
        total['life'] = total.isin(["X"]).sum(axis=1)
        total['life'] = total['life'].apply(lambda x : (7-x) * "🧡")
        total.to_excel(f'{class_id}_{week}주차.xlsx', index=False)

In [31]:
class_dic = {}
class_dic['PA3A'] = 'https://band.us/band/85178180'
class_dic['PB3A'] = 'https://band.us/band/85178672'
class_dic['PD3A'] = 'https://band.us/band/85186142'
class_dic['PJ3A'] = 'https://band.us/band/85186150'
class_dic['PURPLE'] = 'https://band.us/band/85186110'
class_dic['RED'] = 'https://band.us/band/85186132'
## class_dic['PB7A'] = 'https://band.us/band/84600978'
## class_dic['PD7A'] = 'https://band.us/band/84601004'

In [29]:
# 돌리기 
week = 1
comment(5, 2)

In [4]:
p=5
pds = [f'PDS{i}회차' for i in range(1, p+1)]
pds

['PDS1회차', 'PDS2회차', 'PDS3회차', 'PDS4회차', 'PDS5회차']

## 복습테스트 함수

In [ ]:
def re_test(tag):
    
    for class_id in list(class_dic.keys()):
        url = class_dic[class_id] + f'/hashtag/{tag}'
        driver.get(url)


        # 밴드 이름 가져오기 
        band_name = driver.find_element_by_class_name('uriText').text


        # 퀴즈박스 클릭
        driver.find_element_by_class_name('item.-quiz').click() 


        # 퀴즈명 가져오기 
        quiz_name = driver.find_element_by_xpath('//*[@id="wrap"]/div[3]/div/div/section/div[2]/div/section/div/div[3]/div[3]/div[4]/div/div/p[2]')
        quiz_name = quiz_name.text.replace("~", "_")


        scroll = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, '//*[@id="wrap"]/div[3]/div/div/section')))
        driver.execute_script("arguments[0].scrollBy(0,arguments[0].scrollHeight)", scroll) # t스크롤

        driver.find_element_by_class_name('button._btnOpenTakerList').click() # 응답자 보기 클릭
        driver.find_element_by_class_name('btnMoreMenu._btnHeaderMoreMenu').click() # 점 3개 클릭
        driver.find_element_by_class_name('btnMenuText._btnDownloadGradingResult').click() # 채점결과 다운로드 

        time.sleep(2)


        # 저장된 파일 불러오기 
        add = f'C:/Users/hmkang/Downloads/{band_name}_채점결과_{quiz_name}_' + r'{}_{}.xlsx'
        filename = glob.glob(add.format("*", "*"))[0]
        rere = pd.read_excel(filename, header=3)

        # 멤버 및 점수만 뽑아내기 
        rere['멤버'] = rere['멤버'].str.extract(r'([가-힣].*)')
        rere = rere.dropna()
        rere = rere[['멤버', '점수']]
        return rere

In [ ]:
tag = '온택트복습TEST'
re_test(tag)

## Free Rider 함수

In [58]:
class_dic = {}
class_dic['PA3A'] = 'https://band.us/band/85178180'
class_dic['PB3A'] = 'https://band.us/band/85178672'
class_dic['PD3A'] = 'https://band.us/band/85186142'
class_dic['PJ3A'] = 'https://band.us/band/85186150'
class_dic['PURPLE'] = 'https://band.us/band/85186110'
class_dic['RED'] = 'https://band.us/band/85186132'
## class_dic['PB7A'] = 'https://band.us/band/84600978'
## class_dic['PD7A'] = 'https://band.us/band/84601004'

In [80]:
member_list = []
member_count = []

for class_id in list(class_dic.keys()):
    url = class_dic[class_id]+'/member'
    driver.get(url)
    time.sleep(1)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    num = int(soup.select('._memberCount')[0].text)-2
    member_count.append([class_id, num])
    member = soup.select('.ellipsis')
    
    for name in member:
        who = name.text
        who = who.replace('\n', "")
        if class_id == 'PURPLE':
            who = who[5:].strip()
        member_list.append([class_id, who])
        
member12 = pd.DataFrame(member_count, columns=['반명', '학생 수'])

member11 = pd.DataFrame(member_list, columns=['반명', '이름'])
member11['이름']=member11['이름'].str.extract(r'([가-힣].*)')

with pd.ExcelWriter('1주차_memeber.xlsx') as writer:  
    member11.to_excel(writer, sheet_name='1주차_학생명단', index=False)
    member12.to_excel(writer, sheet_name='1주차_학생수', index=False)

In [78]:
num = int(soup.select('._memberCount')[0].text)
num-2

29

# excel  불러와서 한 시트에 합치기
하지만 나중에는 처음부터 따로 저장도 가능...

In [93]:
week_1 = {}
for class_id in list(class_dic.keys()):
    week_1[f'{class_id}'] = pd.read_excel(f'{class_id}_1주차.xlsx')

In [97]:
week_1.keys()

dict_keys(['PA3A', 'PB3A', 'PD3A', 'PJ3A', 'PURPLE', 'RED'])

In [104]:
with pd.ExcelWriter('1주차_comment.xlsx') as writer:  
    for class_id in week_1.keys():
        week_1[class_id].to_excel(writer, sheet_name=f'{class_id}', index=False)s

In [107]:
df = pd.read_excel('1주차_comment.xlsx', sheet_name = 'RED')

In [108]:
df

,name,pds1회차,pds2회차,pds3회차,pds4회차,단어시험01회,단어시험02회,life
0,오지현,O,O,O,O,49,49,🧡🧡🧡🧡🧡🧡🧡
1,김도현,O,O,O,O,48,50,🧡🧡🧡🧡🧡🧡🧡
2,김민정,O,O,O,O,48,48,🧡🧡🧡🧡🧡🧡🧡
3,김예진,O,O,O,O,45,45,🧡🧡🧡🧡🧡🧡🧡
4,김지희,O,O,O,O,45,45,🧡🧡🧡🧡🧡🧡🧡
5,박수현,O,O,O,O,50,50,🧡🧡🧡🧡🧡🧡🧡
6,서유빈,O,O,O,O,47,49,🧡🧡🧡🧡🧡🧡🧡
7,서지원,O,O,O,O,42,44,🧡🧡🧡🧡🧡🧡🧡
8,손여빈,O,O,O,O,49,50,🧡🧡🧡🧡🧡🧡🧡
9,송예윤,O,O,O,O,47,47,🧡🧡🧡🧡🧡🧡🧡


In [115]:
pd.merge(week_1['PA3A'], week_1['PB3A'], on='name', how='outer') 
# 데이터프레임 자체를 딕셔너리 형태로 저장해서 그것을 꺼내서 사용 가능하다.... 

,name,pds1회차_x,pds2회차_x,pds3회차_x,pds4회차_x,pds5회차_x,단어시험01회_x,단어시험02회_x,life_x,pds1회차_y,pds2회차_y,pds3회차_y,pds4회차_y,pds5회차_y,단어시험01회_y,단어시험02회_y,life_y
0,강혜린,O,O,O,O,O,50,50,🧡🧡🧡🧡🧡🧡🧡,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,구준회,O,O,O,O,O,X,X,🧡🧡🧡🧡🧡,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,권영진,O,O,O,O,O,49,49,🧡🧡🧡🧡🧡🧡🧡,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,권지훈,X,X,X,X,X,50,50,🧡🧡,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,김성준,O,O,O,O,O,X,X,🧡🧡🧡🧡🧡,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,정채원,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O,O,O,O,O,46,42,🧡🧡🧡🧡🧡🧡🧡
70,최준혁,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O,O,O,O,O,45,46,🧡🧡🧡🧡🧡🧡🧡
71,최진성,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O,O,O,O,O,X,X,🧡🧡🧡🧡🧡
72,홍샘,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,X,O,O,O,X,X,🧡🧡🧡


In [116]:
week_1['PA3A'].name

0       강혜린
1       구준회
2       권영진
3       권지훈
4       김성준
5     김영현28
6       김채원
7       나소현
8       노건우
9       노재선
10      문하진
11      박민아
12      박상은
13      박성현
14      백영건
15      송원진
16      안경수
17      안민수
18      안정현
19      유승민
20      윤준혁
21      이경민
22      이규희
23      이다은
24      이동현
25      이수진
26      이예은
27      이준헌
28      이형준
29      이혜빈
30      임선우
31      임채린
32      장지원
33      전승범
34      정윤혜
35      정진형
36       채민
37      최진웅
38      추예진
39      한민족
40      홍혜원
41      김예림
42      신태웅
Name: name, dtype: object

## pds 함수 (옛날거)

In [40]:
def pds(nums):
    
    field = 'PDS복습'
    
    for class_id in list(class_dic.keys()):
        
        student_data = []
        
        for num in nums:
            
            tagnum = f'{num}차'

            tag = f'{field}{tagnum}'

            url = class_dic[class_id] + f'/hashtag/{tag}'
            driver.get(url)
            time.sleep(1)

            try: 
                driver.find_element_by_class_name('commentToggle._commentToggleBtn').click()
                time.sleep(2)

                for _ in range(4):
                    try:
                        driver.find_element_by_class_name('prevComment').click()
                        time.sleep(0.2)
                    except :
                        pass



                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                students = soup.select('.itemWrap')


                for student in students :

                    name = student.select('button > strong')[0].text
                    if class_id == 'PURPLE':    # 퍼플은 반명을 한글로 적는 바람에 따로 해줘야 함 
                        name = name[5:].strip() # 전부 다 이거로 하게 되면 띄어쓰기 없는 애들은 솔팅 안됨 
                    score = 'O'

                    student_data.append([class_id, field, tagnum, name, score])

                print(len(student_data))
            
            except:
                pass

        columns = ['반명', '숙제이름', '회차', '이름', '점수']
        student_list = pd.DataFrame(student_data, columns=columns)
        student_list = student_list.drop_duplicates()
        student_list['이름'] = student_list['이름'].str.extract(r'([가-힣].*)') # 한글로 시작하는 시점부터 뒤까지 전부 불러와라 
        student_list.to_excel(f'{class_id}_{field}.xlsx', index=False)

In [ ]:
nums = list(range(1, 14))
pds(nums)

In [5]:
v = 2
voca = [f'단어시험0{i}회' for i in range(1, v+1)]
voca

['단어시험01회', '단어시험02회']

In [ ]:
students

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
students = soup.select('.itemWrap')


In [ ]:
data = []
for student in students :
    name = student.select('button > strong')[0].text
    if class_id == 'PURPLE':
        name = name[5:].strip()
    try :
        score = student.select('div.commentBody._commentModification > p')[0].text
    except :
        score =""
    data.append([name, score])
data

In [ ]:
students

In [ ]:
name = student.select('button > strong')[0].text
name

In [ ]:
score = student.select('p')[0].text

In [ ]:
students

In [ ]:
a = driver.find_element_by_css_selector('div.itemWrap > div.writeInfo > button > strong')

In [ ]:
student

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')


In [ ]:
pds = soup.select('div.itemWrap > div.writeInfo > button')

In [ ]:
name_list = []
# pds_n = input('태그를 입력하세요 :')
pds_n = 'PDS복습34차'

for name in pds: 
    who = name.text
    who = who.replace('\n', "")
    where, who = who.split(" ")
    what = pds_n

    name_list.append([what, where, who])

    
df = pd.DataFrame(name_list, columns=['숙제이름', '반명', '이름'])
df

In [ ]:
class_dic

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')




In [ ]:
# class_number = input("반명은? : ")
url = class_dic[class_number]+'/member'
# driver.get(url)
a = list(class_dic.keys())
a

In [ ]:
class_dic = {}
class_dic['PA3A'] = 'https://band.us/band/84576914'
class_dic['PB3A'] = 'https://band.us/band/84576918'
class_dic['PD3A'] = 'https://band.us/band/84576926'
class_dic['PJ3A'] = 'https://band.us/band/84576928'
class_dic['PURPLE'] = 'https://band.us/band/84570800'
class_dic['RED'] = 'https://band.us/band/84576846'
class_dic['PB7A'] = 'https://band.us/band/84600978'
class_dic['PD7A'] = 'https://band.us/band/84601004'


# class_number = input("반명은? : ")
member_list = []
for class_number in list(class_dic.keys()):
    url = class_dic[class_number]+'/member'
    driver.get(url)
    time.sleep(2)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    member = soup.select('span.text.-flex')

    for name in member[1:]:
        who = name.text
        who = who.replace('\n', "")
        if class_number == 'PURPLE':
            who = who[5:].strip()
        where = class_number
        member_list.append([where, who])

member11 = pd.DataFrame(member_list, columns=['반명', '이름'])
member11['이름']=member11['이름'].str.extract(r'([가-힣].*)')
member11.to_excel('student_08.xlsx', index=False)

In [ ]:
member11['이름']=member11['이름'].str.extract(r'([가-힣].*)')

member11

In [ ]:
member11.to_excel('student_08.xlsx', index=False)

In [ ]:
class_dic = {}
class_dic['PA3A'] = 'https://band.us/band/84576914'
class_dic['PB3A'] = 'https://band.us/band/84576918'
class_dic['PD3A'] = 'https://band.us/band/84576926'
class_dic['PJ3A'] = 'https://band.us/band/84576928'
class_dic['PURPLE'] = 'https://band.us/band/84570800'
class_dic['RED'] = 'https://band.us/band/84576846'
class_dic['PB7A'] = 'https://band.us/band/84600978'
class_dic['PD7A'] = 'https://band.us/band/84601004'


In [ ]:
driver.add

In [ ]:
s = requests.Session()

In [ ]:
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'
}
s.headers.update(headers)

In [ ]:
for cookie in driver.get_cookies():
    c = {cookie['name'] : cookie['value']}
    s.cookies.update(c)

In [ ]:
s.cookies


In [ ]:
response = s.get('https://band.us/band/84576914/hashtag/PDS%EB%B3%B5%EC%8A%B532%EC%B0%A8')
soup = BeautifulSoup(response.text, 'lxml')
soup

In [ ]:
for cookie in driver.get_cookies():
    driver.add_cookie(c)

In [ ]:
cks = driver.get_cookies()
cks